In [1]:
import pandas as pd
import numpy as np
import os
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

#### Declaring functions to find missing values, correlation , encoding categorical variables etc

In [2]:
## Missing values###########
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns
###############################

In [3]:
#Encoding categorical variables
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
def encoding_cat_vars(df,le):
    le_count = 0
    one_hot_count = 0
    # Iterate through the columns
    for col in df:
        if df[col].dtype == 'object' or df[col].dtype =='bool' :
            # If 2 or fewer unique categories
            if len(list(df[col].unique())) <= 2:
                # Train on the training data
                le.fit(df[col])
                # Transform both training and testing data
                df[col] = le.transform(df[col])    
                # Keep track of how many columns were label encoded
                le_count += 1
            else:
                one_hot_count +=1
    print ("no of columns label encoded : "+str(le_count))
    #One Hot encoding
    df = pd.get_dummies(df)
    print ("no of columns one hot encoded encoded : "+str(one_hot_count))
    return df

In [5]:
# POS_CASH_balance = pd.read_csv(os.getcwd()+'\\POS_CASH_balance.csv')
POS_CASH_balance = pd.read_csv('POS_CASH_balance.csv')

In [6]:
#Removing highly correlated field
corr_matrix = POS_CASH_balance.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.85)]
to_drop

['CNT_INSTALMENT_FUTURE']

In [7]:
#Check for missing values
missing_values = missing_values_table(POS_CASH_balance)
missing_values['column_name'] = missing_values.index
missing_values = missing_values.reset_index(drop = True)
missing_values

Your selected dataframe has 8 columns.
There are 2 columns that have missing values.


,Missing Values,% of Total Values,column_name
0,26087,0.3,CNT_INSTALMENT_FUTURE
1,26071,0.3,CNT_INSTALMENT


In [8]:
POS_CASH_balance[POS_CASH_balance['SK_ID_CURR']==363914].sort_values(by=['MONTHS_BALANCE'],ascending=False).head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
336004,2627928,363914,-1,24.0,17.0,Active,0,0
4912537,2627928,363914,-2,24.0,18.0,Active,0,0
3380235,2627928,363914,-3,24.0,19.0,Active,0,0
3561428,2627928,363914,-4,24.0,20.0,Active,0,0
7227578,2627928,363914,-5,24.0,21.0,Active,0,0


In [9]:
prev = pd.read_csv('previous_application.csv')

In [10]:
POS_CASH_balance = pd.merge(POS_CASH_balance,prev[['SK_ID_PREV','CNT_PAYMENT']],how='left',on=['SK_ID_PREV'])

In [11]:
POS_CASH_balance.shape

(10001358, 9)

## IMPUTING nan values

#### NAME_CONTRACT_STATUS == Active

* Removing SK_ID_PREV = 1531600, 1697740, 1519880, 2580175
* in this 1531600 cannot be imputed as it is full of null values
* Same to be deleted from prev_apps

In [12]:
def cnt_future(sk_ids):
    for sk_id in sk_ids:
        indices = POS_CASH_balance[POS_CASH_balance['SK_ID_PREV']==sk_id].sort_values(by=['MONTHS_BALANCE']).index
        print('Null value indices {} for SK_PREV_ID {} '.format(list(indices),sk_id))
        
        base_mnth = POS_CASH_balance.at[indices[0],'MONTHS_BALANCE']
#         print(base_mnth)
        for ind in indices:
            CNT_INSTALMENT_FUTURE = POS_CASH_balance.at[ind,'CNT_PAYMENT'] + base_mnth - POS_CASH_balance.at[ind,'MONTHS_BALANCE'] 
#             print(CNT_INSTALMENT_FUTURE)
            POS_CASH_balance.at[ind,'CNT_INSTALMENT_FUTURE'] = CNT_INSTALMENT_FUTURE

            CNT_INSTALMENT = POS_CASH_balance.at[ind,'CNT_PAYMENT']
#             print(CNT_INSTALMENT)
            POS_CASH_balance.at[ind,'CNT_INSTALMENT'] = CNT_INSTALMENT 
            

In [13]:
POS_CASH_balance[(POS_CASH_balance['SK_ID_PREV'].isin([1531600,1697740, 1519880, 2580175]))].sort_values(by=['SK_ID_PREV', 'MONTHS_BALANCE'])

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF,CNT_PAYMENT
776482,1519880,322459,-23,NaN,NaN,Active,0,0,12.0
5290717,1519880,322459,-22,1.0,0.0,Active,0,0,12.0
1272090,1519880,322459,-21,2.0,0.0,Active,0,0,12.0
4699888,1519880,322459,-20,3.0,0.0,Active,0,0,12.0
4958927,1519880,322459,-19,4.0,0.0,Active,0,0,12.0
1055842,1519880,322459,-18,5.0,0.0,Active,0,0,12.0
4264774,1519880,322459,-17,6.0,0.0,Active,0,0,12.0
8538424,1519880,322459,-16,7.0,0.0,Active,0,0,12.0
1588663,1519880,322459,-15,8.0,0.0,Active,0,0,12.0
776933,1519880,322459,-14,9.0,0.0,Active,0,0,12.0


In [14]:
cnt_future([1531600, 1697740, 1519880, 2580175])

Null value indices [5018718, 8214569, 5239791, 4695804, 2316683, 4732751, 1978546, 2482106, 5109964, 1188603, 8472131, 4365324, 2106335, 5568375, 6836182, 2211599, 4504671, 647938, 211849, 1732420, 480582, 3321317, 3355859, 1747234, 248875, 614950, 2558953] for SK_PREV_ID 1531600 
Null value indices [2916928, 3539278, 3532478, 4786264, 756349, 4471361, 2575444, 5152419, 2099256] for SK_PREV_ID 1697740 
Null value indices [776482, 5290717, 1272090, 4699888, 4958927, 1055842, 4264774, 8538424, 1588663, 776933, 3549769, 922163, 33796] for SK_PREV_ID 1519880 
Null value indices [4215727, 4396415, 254456, 4721571, 4210814, 578545, 4657526, 5986955, 5022339, 4282619, 1315688, 346072, 1019539, 1539554, 2262554, 3824691, 693888, 1380330, 3499895, 160653, 5068361, 5729694, 5630131, 6120362] for SK_PREV_ID 2580175 


#### NAME_CONTRACT_STATUS == Xna


* removing only those 2 rows [1711630, 1378042]

In [16]:
POS_CASH_balance[POS_CASH_balance['NAME_CONTRACT_STATUS'] == 'XNA']

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF,CNT_PAYMENT
1614508,1378042,142208,-21,36.0,36.0,XNA,0,0,36.0
1614509,1711630,309000,-24,36.0,36.0,XNA,0,0,36.0


In [17]:
cnt_future([1711630, 1378042])

Null value indices [1614509, 1054869, 4885361, 5900467, 4686172, 3058984, 1620635, 2533978, 32097, 1575386, 1488786, 4960806, 3545284, 4181655, 321802, 1057355, 3808543, 6755937] for SK_PREV_ID 1711630 
Null value indices [1614508, 1054868, 4885360, 5900466, 4686171, 3058983, 1620634, 2533977, 32096, 1575385, 1488785, 4960805, 3545283, 4181654, 321801, 1057354, 3808542, 6755936] for SK_PREV_ID 1378042 


* checking for CNT_INSTALMENT_FUTURE > CNT_INSTALMENT

In [18]:
# POS_CASH_balance[POS_CASH_balance['CNT_INSTALMENT_FUTURE']>POS_CASH_balance['CNT_INSTALMENT']]

#### NAME_CONTRACT_STATUS == Canceled

In [20]:
def cnt_future_cancel(sk_ids):
    multi_null_ids = []
    for sk_id in sk_ids:
        indices = POS_CASH_balance[(POS_CASH_balance['SK_ID_PREV']==sk_id) & (POS_CASH_balance['CNT_INSTALMENT'].isnull())].sort_values(by=['MONTHS_BALANCE']).index
 #       print('Null value indices {} for SK_PREV_ID {} '.format(list(indices),sk_id))
        base_mnth = POS_CASH_balance.at[indices[0],'MONTHS_BALANCE']
#         print(base_mnth)
        if len(indices)==1:
            CNT_INSTALMENT_FUTURE = POS_CASH_balance.at[indices[0],'CNT_PAYMENT'] + base_mnth - POS_CASH_balance.at[indices[0],'MONTHS_BALANCE'] 
#             print(CNT_INSTALMENT_FUTURE)
            POS_CASH_balance.at[indices[0],'CNT_INSTALMENT_FUTURE'] = CNT_INSTALMENT_FUTURE

            CNT_INSTALMENT = POS_CASH_balance.at[indices[0],'CNT_PAYMENT']
#             print(CNT_INSTALMENT)
            POS_CASH_balance.at[indices[0],'CNT_INSTALMENT'] = CNT_INSTALMENT 
        else:
            multi_null_ids.append(sk_id)
    return multi_null_ids
            

In [21]:
null_skid = POS_CASH_balance[(POS_CASH_balance['NAME_CONTRACT_STATUS']=='Canceled')&(POS_CASH_balance['CNT_INSTALMENT'].isnull())]['SK_ID_PREV'].values

In [22]:
null_skid

array([2200360, 1720362, 2239989, 2134263, 2122901, 1287451, 1176218,
       1546576, 1976128, 1764746, 1160113], dtype=int64)

In [23]:
multi_null_list_cncl = cnt_future_cancel(null_skid)

In [24]:
multi_null_list_cncl

[2200360, 2134263, 1160113]

In [25]:
POS_CASH_balance[POS_CASH_balance['SK_ID_PREV'].isin(multi_null_list_cncl)].sort_values(by=['SK_ID_PREV','MONTHS_BALANCE'])

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF,CNT_PAYMENT
8559422,1160113,236646,-4,NaN,NaN,Approved,0,0,36.0
7042326,1160113,236646,-3,NaN,NaN,Approved,0,0,36.0
9856282,1160113,236646,-2,NaN,NaN,Canceled,0,0,36.0
7556193,2134263,366250,-8,NaN,NaN,Canceled,0,0,12.0
7803396,2134263,366250,-7,NaN,NaN,Returned to the store,0,0,12.0
9277737,2134263,366250,-6,12.0,12.0,Active,0,0,12.0
8207099,2134263,366250,-5,12.0,11.0,Active,0,0,12.0
7324893,2134263,366250,-4,12.0,10.0,Active,0,0,12.0
9010783,2134263,366250,-3,12.0,8.0,Active,0,0,12.0
9653299,2134263,366250,-2,12.0,7.0,Active,0,0,12.0


## Adding default rate of each contract status

In [58]:
POS_CASH_balance['NAME_CONTRACT_STATUS'].unique()

array(['Active', 'Completed', 'Approved', 'Signed', 'Demand',
       'Amortized debt', 'Returned to the store', 'Canceled', 'XNA'],
      dtype=object)

In [60]:
# app_train = pd.read_csv('application_train.csv')[['SK_ID_CURR','TARGET']]
app_train = pd.read_csv('application_train.csv')[['SK_ID_CURR','TARGET']]

In [61]:
POS_CASH_balance = pd.merge(POS_CASH_balance,app_train,how='left',on=['SK_ID_CURR'])

* Finding the mean between CNT_INSTALMENT and CNT_INSTALMENT_FUTURE and aggregating on SK_ID
* added by ebby on 20th Aug

In [62]:
POS_CASH_balance['CNT_DIFF'] = POS_CASH_balance['CNT_INSTALMENT'] - POS_CASH_balance['CNT_INSTALMENT_FUTURE']

In [63]:
pos_cnt_diff_prev_mean = POS_CASH_balance.groupby(['SK_ID_PREV','SK_ID_CURR'])[['CNT_DIFF']].mean().rename(columns={'CNT_DIFF':'POS_CNT_DIFF_PREV_MEAN'}).reset_index()

In [64]:
pos_cnt_diff_curr_mean = pos_cnt_diff_prev_mean.groupby(['SK_ID_CURR'])[['POS_CNT_DIFF_PREV_MEAN']].mean().rename(columns={'POS_CNT_DIFF_PREV_MEAN':'POS_CNT_DIFF_MEAN'}).reset_index()
pos_cnt_diff_curr_min = pos_cnt_diff_prev_mean.groupby(['SK_ID_CURR'])[['POS_CNT_DIFF_PREV_MEAN']].min().rename(columns={'POS_CNT_DIFF_PREV_MEAN':'POS_CNT_DIFF_MIN'}).reset_index()
pos_cnt_diff_curr_max = pos_cnt_diff_prev_mean.groupby(['SK_ID_CURR'])[['POS_CNT_DIFF_PREV_MEAN']].max().rename(columns={'POS_CNT_DIFF_PREV_MEAN':'POS_CNT_DIFF_MAX'}).reset_index()
pos_cnt_diff_curr_var = pos_cnt_diff_prev_mean.groupby(['SK_ID_CURR'])[['POS_CNT_DIFF_PREV_MEAN']].var().rename(columns={'POS_CNT_DIFF_PREV_MEAN':'POS_CNT_DIFF_VAR'}).reset_index()
pos_cnt_diff_curr_var.fillna(value=0, inplace=True)

In [65]:
pos_status = POS_CASH_balance.groupby(['NAME_CONTRACT_STATUS','TARGET'])[['SK_ID_CURR']].count().reset_index()
pos_status = pos_status.pivot(index='NAME_CONTRACT_STATUS', columns='TARGET', values='SK_ID_CURR').reset_index()
pos_status = pos_status.rename(columns={0:'POS_STS_CORRECT',1:'POS_STS_DEFAULT'})
pos_status.head()

TARGET,NAME_CONTRACT_STATUS,POS_STS_CORRECT,POS_STS_DEFAULT
0,Active,7244503.0,574068.0
1,Amortized debt,240.0,125.0
2,Approved,3902.0,319.0
3,Canceled,10.0,2.0
4,Completed,587918.0,46952.0


* made a change in denominator

In [66]:
POS_CASH_balance = pd.merge(POS_CASH_balance,pos_status,on=['NAME_CONTRACT_STATUS'],how='left')

In [67]:
# POS_CASH_balance['DEFAULT_RATE_POS_STATUS'] = POS_CASH_balance['POS_STS_DEFAULT']/POS_CASH_balance['POS_STS_CORRECT']
POS_CASH_balance['DEFAULT_RATE_POS_STATUS'] = POS_CASH_balance['POS_STS_DEFAULT']/(POS_CASH_balance['POS_STS_DEFAULT']+POS_CASH_balance['POS_STS_CORRECT'])
POS_CASH_balance = POS_CASH_balance.drop(['POS_STS_DEFAULT','POS_STS_CORRECT'], axis=1)

In [68]:
POS_CASH_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF,TARGET,CNT_DIFF,DEFAULT_RATE_POS_STATUS
0,1000001,158271,-10,12.0,12.0,Active,0,0,0.0,0.0,0.073424
1,1000001,158271,-9,12.0,11.0,Active,0,0,0.0,1.0,0.073424
2,1000001,158271,-8,2.0,0.0,Completed,0,0,0.0,2.0,0.073955
3,1000002,101962,-54,4.0,4.0,Active,0,0,NaN,0.0,0.073424
4,1000002,101962,-53,6.0,3.0,Active,0,0,NaN,3.0,0.073424


In [69]:
POS_CASH_balance[POS_CASH_balance['DEFAULT_RATE_POS_STATUS'].isnull()]

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF,TARGET,CNT_DIFF,DEFAULT_RATE_POS_STATUS
2183067,1378042,142208,-21,36.0,36.0,XNA,0,0,0.0,0.0,NaN
4026739,1711630,309000,-24,36.0,36.0,XNA,0,0,0.0,0.0,NaN


In [70]:
app_test = pd.read_csv('application_test.csv')

In [71]:
app_test[app_test['SK_ID_CURR']==309000]

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR


#### Above missing SK_ID_CURR are not present in Test file

In [72]:
POS_CASH_balance = POS_CASH_balance.dropna(subset=['DEFAULT_RATE_POS_STATUS'])
POS_CASH_balance.shape

(10001331, 11)

#### Default rate per status is added to the dataframe

#### Find Month size

In [73]:
POS_CASH_balance_month = POS_CASH_balance.groupby(['SK_ID_CURR'])['MONTHS_BALANCE'].size().reset_index()
POS_CASH_balance_month = POS_CASH_balance_month.rename(columns={'MONTHS_BALANCE':'SIZE_POS_MONTH_BAL'})
POS_CASH_balance_month.head(2)

,SK_ID_CURR,SIZE_POS_MONTH_BAL
0,100001,9
1,100002,19


In [74]:
POS_CASH_balance_sts = POS_CASH_balance[['SK_ID_CURR','NAME_CONTRACT_STATUS']].rename(columns={'NAME_CONTRACT_STATUS':'POS_STATUS'})
POS_CASH_balance_sts_sum = POS_CASH_balance_sts.add_prefix('SUM_').rename(columns={'SUM_SK_ID_CURR':'SK_ID_CURR'})
POS_CASH_balance_sts_mean = POS_CASH_balance_sts.add_prefix('MEAN_').rename(columns={'MEAN_SK_ID_CURR':'SK_ID_CURR'})
POS_CASH_balance_sts_sum = encoding_cat_vars(POS_CASH_balance_sts_sum,le)
POS_CASH_balance_sts_mean = encoding_cat_vars(POS_CASH_balance_sts_mean,le)
POS_CASH_balance_sts_sum = POS_CASH_balance_sts_sum.groupby(['SK_ID_CURR']).sum().reset_index()
POS_CASH_balance_sts_mean = POS_CASH_balance_sts_mean.groupby(['SK_ID_CURR']).mean().reset_index()
POS_CASH_balance_sts = pd.merge(POS_CASH_balance_sts_mean,POS_CASH_balance_sts_sum,on=['SK_ID_CURR'],how='inner')
POS_CASH_balance_sts.head(2)

no of columns label encoded : 0
no of columns one hot encoded encoded : 1
no of columns label encoded : 0
no of columns one hot encoded encoded : 1


,SK_ID_CURR,MEAN_POS_STATUS_Active,MEAN_POS_STATUS_Amortized debt,MEAN_POS_STATUS_Approved,MEAN_POS_STATUS_Canceled,MEAN_POS_STATUS_Completed,MEAN_POS_STATUS_Demand,MEAN_POS_STATUS_Returned to the store,MEAN_POS_STATUS_Signed,SUM_POS_STATUS_Active,SUM_POS_STATUS_Amortized debt,SUM_POS_STATUS_Approved,SUM_POS_STATUS_Canceled,SUM_POS_STATUS_Completed,SUM_POS_STATUS_Demand,SUM_POS_STATUS_Returned to the store,SUM_POS_STATUS_Signed
0,100001,0.777778,0.0,0.0,0.0,0.222222,0.0,0.0,0.0,7.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
1,100002,1.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [75]:
POS_CASH_balance_cal = POS_CASH_balance[['SK_ID_CURR','CNT_INSTALMENT','CNT_INSTALMENT_FUTURE',
                                    'SK_DPD','SK_DPD_DEF']]
POS_CASH_balance_sum = POS_CASH_balance_cal.add_prefix('SUM_').rename(columns={'SUM_SK_ID_CURR':'SK_ID_CURR'})
POS_CASH_balance_max = POS_CASH_balance_cal.add_prefix('MAX_').rename(columns={'MAX_SK_ID_CURR':'SK_ID_CURR'})
POS_CASH_balance_mean = POS_CASH_balance_cal.add_prefix('MEAN_').rename(columns={'MEAN_SK_ID_CURR':'SK_ID_CURR'})
POS_CASH_balance_min = POS_CASH_balance_cal.add_prefix('MIN_').rename(columns={'MIN_SK_ID_CURR':'SK_ID_CURR'})
POS_CASH_balance_var = POS_CASH_balance_cal.add_prefix('VAR_').rename(columns={'VAR_SK_ID_CURR':'SK_ID_CURR'})


In [76]:
POS_CASH_balance_sum = POS_CASH_balance_sum.groupby(['SK_ID_CURR']).sum().reset_index()
POS_CASH_balance_max = POS_CASH_balance_max.groupby(['SK_ID_CURR']).max().reset_index()
POS_CASH_balance_mean = POS_CASH_balance_mean.groupby(['SK_ID_CURR']).mean().reset_index()
POS_CASH_balance_min = POS_CASH_balance_min.groupby(['SK_ID_CURR']).min().reset_index()
POS_CASH_balance_var = POS_CASH_balance_var.groupby(['SK_ID_CURR']).var().reset_index()

In [77]:
POS_CASH_balance_mean = POS_CASH_balance_mean.drop(['MEAN_CNT_INSTALMENT','MEAN_CNT_INSTALMENT_FUTURE'], axis=1)
POS_CASH_balance_max = POS_CASH_balance_max.drop(['MAX_CNT_INSTALMENT_FUTURE'], axis=1)
POS_CASH_balance_sum = POS_CASH_balance_sum.drop(['SUM_CNT_INSTALMENT','SUM_CNT_INSTALMENT_FUTURE'], axis=1 )
POS_CASH_balance_min = POS_CASH_balance_min.drop(['MIN_CNT_INSTALMENT','MIN_SK_DPD','MIN_SK_DPD_DEF'], axis=1)
POS_CASH_balance_var = POS_CASH_balance_var.drop(['VAR_CNT_INSTALMENT','VAR_CNT_INSTALMENT_FUTURE'], axis=1)

In [78]:
POS_CASH_balance_cal = pd.merge(POS_CASH_balance_mean,POS_CASH_balance_max,on=['SK_ID_CURR'],how='inner')
POS_CASH_balance_cal = pd.merge(POS_CASH_balance_cal,POS_CASH_balance_sum,on=['SK_ID_CURR'],how='inner')
POS_CASH_balance_cal = pd.merge(POS_CASH_balance_cal,POS_CASH_balance_min,on=['SK_ID_CURR'],how='inner')
POS_CASH_balance_cal = pd.merge(POS_CASH_balance_cal,POS_CASH_balance_var,on=['SK_ID_CURR'],how='inner')
POS_CASH_balance_cal.head()

,SK_ID_CURR,MEAN_SK_DPD,MEAN_SK_DPD_DEF,MAX_CNT_INSTALMENT,MAX_SK_DPD,MAX_SK_DPD_DEF,SUM_SK_DPD,SUM_SK_DPD_DEF,MIN_CNT_INSTALMENT_FUTURE,VAR_SK_DPD,VAR_SK_DPD_DEF
0,100001,0.777778,0.777778,4.0,7,7,7,7,0.0,5.444444,5.444444
1,100002,0.000000,0.000000,24.0,0,0,0,0,6.0,0.000000,0.000000
2,100003,0.000000,0.000000,12.0,0,0,0,0,0.0,0.000000,0.000000
3,100004,0.000000,0.000000,4.0,0,0,0,0,0.0,0.000000,0.000000
4,100005,0.000000,0.000000,12.0,0,0,0,0,0.0,0.000000,0.000000


In [79]:
POS_CASH_balance_rate = POS_CASH_balance[['SK_ID_CURR','DEFAULT_RATE_POS_STATUS']]
POS_CASH_balance_rate_mean = POS_CASH_balance_rate.groupby(['SK_ID_CURR']).mean().reset_index().rename(columns={'DEFAULT_RATE_POS_STATUS':'MEAN_DEFAULT_RATE_POS_STATUS'})
POS_CASH_balance_rate_max = POS_CASH_balance_rate.groupby(['SK_ID_CURR']).max().reset_index().rename(columns={'DEFAULT_RATE_POS_STATUS':'MAX_DEFAULT_RATE_POS_STATUS'})
POS_CASH_balance_rate_sum = POS_CASH_balance_rate.groupby(['SK_ID_CURR']).max().reset_index().rename(columns={'DEFAULT_RATE_POS_STATUS':'SUM_DEFAULT_RATE_POS_STATUS'})
POS_CASH_balance_rate = pd.merge(POS_CASH_balance_rate_mean,POS_CASH_balance_rate_max,on=['SK_ID_CURR'],how='inner')
POS_CASH_balance_rate = pd.merge(POS_CASH_balance_rate,POS_CASH_balance_rate_sum,on=['SK_ID_CURR'],how='inner')

In [80]:
POS_CASH_balance = pd.merge(POS_CASH_balance_rate,POS_CASH_balance_cal,on=['SK_ID_CURR'],how='inner')
POS_CASH_balance = pd.merge(POS_CASH_balance,POS_CASH_balance_sts,on=['SK_ID_CURR'],how='inner')
POS_CASH_balance = pd.merge(POS_CASH_balance,POS_CASH_balance_month,on=['SK_ID_CURR'],how='inner')


In [81]:
POS_CASH_balance = pd.merge(POS_CASH_balance,pos_cnt_diff_curr_mean,on=['SK_ID_CURR'],how='inner')
POS_CASH_balance = pd.merge(POS_CASH_balance,pos_cnt_diff_curr_min,on=['SK_ID_CURR'],how='inner')
POS_CASH_balance = pd.merge(POS_CASH_balance,pos_cnt_diff_curr_max,on=['SK_ID_CURR'],how='inner')
POS_CASH_balance = pd.merge(POS_CASH_balance,pos_cnt_diff_curr_var,on=['SK_ID_CURR'],how='inner')

In [82]:
POS_CASH_balance.shape

(337252, 35)

In [83]:
#Check for missing values
missing_values = missing_values_table(POS_CASH_balance)
missing_values['column_name'] = missing_values.index
missing_values = missing_values.reset_index(drop = True)
missing_values

Your selected dataframe has 35 columns.
There are 2 columns that have missing values.


,Missing Values,% of Total Values,column_name
0,372,0.1,VAR_SK_DPD
1,372,0.1,VAR_SK_DPD_DEF


In [84]:
POS_CASH_balance.columns

Index(['SK_ID_CURR', 'MEAN_DEFAULT_RATE_POS_STATUS',
       'MAX_DEFAULT_RATE_POS_STATUS', 'SUM_DEFAULT_RATE_POS_STATUS',
       'MEAN_SK_DPD', 'MEAN_SK_DPD_DEF', 'MAX_CNT_INSTALMENT', 'MAX_SK_DPD',
       'MAX_SK_DPD_DEF', 'SUM_SK_DPD', 'SUM_SK_DPD_DEF',
       'MIN_CNT_INSTALMENT_FUTURE', 'VAR_SK_DPD', 'VAR_SK_DPD_DEF',
       'MEAN_POS_STATUS_Active', 'MEAN_POS_STATUS_Amortized debt',
       'MEAN_POS_STATUS_Approved', 'MEAN_POS_STATUS_Canceled',
       'MEAN_POS_STATUS_Completed', 'MEAN_POS_STATUS_Demand',
       'MEAN_POS_STATUS_Returned to the store', 'MEAN_POS_STATUS_Signed',
       'SUM_POS_STATUS_Active', 'SUM_POS_STATUS_Amortized debt',
       'SUM_POS_STATUS_Approved', 'SUM_POS_STATUS_Canceled',
       'SUM_POS_STATUS_Completed', 'SUM_POS_STATUS_Demand',
       'SUM_POS_STATUS_Returned to the store', 'SUM_POS_STATUS_Signed',
       'SIZE_POS_MONTH_BAL', 'POS_CNT_DIFF_MEAN', 'POS_CNT_DIFF_MIN',
       'POS_CNT_DIFF_MAX', 'POS_CNT_DIFF_VAR'],
      dtype='object')

In [85]:
POS_CASH_balance = POS_CASH_balance.drop(columns=['VAR_SK_DPD','VAR_SK_DPD_DEF'])

In [86]:
POS_CASH_balance = POS_CASH_balance.dropna(subset=['MAX_CNT_INSTALMENT','MIN_CNT_INSTALMENT_FUTURE'])

In [87]:
POS_CASH_balance.shape

(337252, 33)

In [88]:
POS_CASH_balance.to_csv('tmp_pos_cash_balance.csv',index=False)